In [1]:
from pathlib import Path
import numpy as np
import math
from itertools import groupby
import h5py
import numpy as np
import unicodedata
import cv2
import time
import timm
import random
from collections import OrderedDict
import pandas as pd
import os
import glob
from PIL import Image
import string
import json
import data_augmentation as da
import json
import os
import numpy as np
from load_washington import *

In [2]:
source = "/home/mhamdan/seq2seqAttenHTR/Transformer_ocr/raw/RIMES"
pt_path = os.path.join(source, "rwth_split")


In [3]:
pt_path

'/home/mhamdan/seq2seqAttenHTR/Transformer_ocr/raw/RIMES/rwth_split'

In [4]:
paths = {"train": open(os.path.join(pt_path, "groundtruth_training_icdar2011.txt")).read().splitlines(),
         "valid": open(os.path.join(pt_path, "ground_truth_validation_icdar2011.txt")).read().splitlines(),
         "test": open(os.path.join(pt_path, "grount_truth_test_icdar2011.txt")).read().splitlines()}

In [127]:
for i in partitions:
    if i == 'test':
        for line in paths[i]:
            #continue
            #print(pt_path+line)
            break
            #i = i.replace("valid","train")

In [117]:
for k in paths.keys():
    print(k)


train
valid
test


In [15]:
paths['train']

['lot_1/00001_L/00001_L_0_0.tiff Suite',
 'lot_1/00001_L/00001_L_2_0.tiff meilleurs',
 'lot_1/00001_L/00001_L_2_2.tiff marche',
 'lot_1/00001_L/00001_L_2_3.tiff suivre',
 'lot_1/00001_L/00001_L_2_4.tiff pour',
 'lot_1/00002_L/00002_L_0_0.tiff Il',
 'lot_1/00002_L/00002_L_0_1.tiff y',
 'lot_1/00002_L/00002_L_0_2.tiff a',
 'lot_1/00002_L/00002_L_0_3.tiff mois',
 'lot_1/00002_L/00002_L_0_4.tiff le',
 'lot_1/00002_L/00002_L_0_5.tiff Mai',
 'lot_1/00002_L/00002_L_0_7.tiff très',
 'lot_1/00002_L/00002_L_2_0.tiff femme',
 'lot_1/00002_L/00002_L_2_1.tiff qui',
 'lot_1/00002_L/00002_L_2_3.tiff Celle-ci',
 'lot_1/00002_L/00002_L_2_4.tiff a',
 'lot_1/00002_L/00002_L_2_5.tiff donc',
 'lot_1/00002_L/00002_L_2_6.tiff fait',
 "lot_1/00002_L/00002_L_2_7.tiff l'objet",
 "lot_1/00002_L/00002_L_2_8.tiff d'une",
 'lot_1/00002_L/00002_L_2_9.tiff expertise',
 'lot_1/00002_L/00002_L_5_0.tiff elle',
 'lot_1/00002_L/00002_L_5_1.tiff a',
 'lot_1/00002_L/00002_L_5_2.tiff moins',
 'lot_1/00002_L/00002_L_5_3.tiff 

In [61]:
text = "façon"

In [62]:
unicodedata.normalize("NFKD", text).encode("ASCII", "ignore").decode("ASCII")

'facon'

In [23]:
gt_dict = {}
for k, v in paths.items():
    for i in v:
        try:
            vv = i.split()
            gt_dict[vv[0]] = vv[1] 
        except:
            print("yo")

yo
yo
yo


In [5]:
partitions = ['train', 'valid', 'test']
def replace_tokens(text, tokens):
    for t in tokens:
        text = text.replace(t, '')
        
    return text


In [119]:
data = []
count = 0
alphabet = string.ascii_lowercase + string.digits + """ "!#$%&\'()*+,./:;<=>?@[\\]^_`{|}~"""

for i in partitions:
    for line in paths[i]:
        i = i.replace("valid","train")
        try:
            split = line.split("-")
            folder = f"{split[0]}-{split[1]}"
            folder_path = os.path.join(source, "words", split[0], folder)
            for img in glob.glob(folder_path+'/{}*'.format(line)):
                label = gt_dict[img.split("/")[-1].replace(".png","")].lower()
                if len(label)==1:
                    continue
                if label in stopwords:
                    continue
                ntexts = [replace_tokens(text, [t for t in text if t not in alphabet]) for text in label]
                data.append({"file":img, "text":"".join(ntexts), "split":i})    
        except KeyError:
            count+=1
            pass

IndexError: list index out of range

In [32]:
stopwords = """-,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z,a,able,about,across,after,all,almost,also,am,among,an,and,any,are,as,at,be,because,been,but,by,can,cannot,could,dear,did,do,does,either,else,ever,every,for,from,get,got,had,has,have,he,her,hers,him,his,how,however,i,if,in,into,is,it,its,just,least,let,like,likely,may,me,might,most,must,my,neither,no,nor,not,of,off,often,on,only,or,other,our,own,rather,said,say,says,she,should,since,so,some,than,that,the,their,them,then,there,these,they,this,tis,to,too,twas,us,wants,was,we,were,what,when,where,which,while,who,whom,why,will,with,would,yet,you,your""".split(",")

In [38]:
data = []
count = 0
alphabet = string.ascii_lowercase + string.digits + """ "!#$%&\'()*+,./:;<=>?@[\\]^_`{|}~"""

for i in partitions:
    for lines in paths[i]:
        if lines=="":
            continue
        i = i.replace("valid","train")
       # print(i)
        try:
#             split = line.split("/")
#             print(line)
            #print(split)
#             folder = f"{split[0:2]}"
            #print(folder)0
            line = lines.split()[0]
            label = lines.split()[1]
            img_path = os.path.join(source, "words","images",line)
            #print(folder_path)
            label = gt_dict[line].lower()
            if len(label)==1:
                continue
            if label in stopwords:
                continue
#             ntexts = [replace_tokens(text, [t for t in text if t not in alphabet]) for text in label]
            data.append({"file":img_path, "text":label, "split":i})    
        except KeyError:
            count+=1
            pass

In [39]:
len(data)

65749

## 

In [40]:
with open("../raw/rimes_word.json", 'w') as f:
    json.dump(data, f)

In [46]:
len(data)

65749

In [42]:
cc = [i for i in data if i['split']=="train"]

In [43]:
len(cc)

58122

In [44]:
augmented_dir = os.path.join("../raw/", 'rimes_augmented/')

In [57]:
with open("../raw/rimes_word.json") as f:
    data = json.load(f)

In [ ]:
augmented_data = da.augment_data(data, augmented_dir)

starting stage 1


In [48]:
from skimage.transform import resize
from skimage.util import img_as_ubyte

In [49]:
def preprocess(img, input_size):
    """Make the process with the `input_size` to the scale resize"""

    def imread(path):
        img = cv2.imread(path)

#         if len(img.shape) == 3:
#             if img.shape[2] == 4:
#                 trans_mask = img[:, :, 3] == 0
#                 img[trans_mask] = [255, 255, 255, 255]

#             img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        return img

    if isinstance(img, str):
        img = imread(img)

    if isinstance(img, tuple):
        image, boundbox = img
        img = imread(image)

        for i in range(len(boundbox)):
            if isinstance(boundbox[i], float):
                total = len(img) if i < 2 else len(img[0])
                boundbox[i] = int(total * boundbox[i])

        img = np.asarray(img[boundbox[0]:boundbox[1], boundbox[2]:boundbox[3]], dtype=np.uint8)

    wt, ht, _ = input_size
    h, w,_ = img.shape
    f = max((w / wt), (h / ht))
    new_size = (max(min(wt, int(w / f)), 1), max(min(ht, int(h / f)), 1))
    img = cv2.resize(img, new_size)

    target = np.ones([ht, wt,3], dtype=np.uint8) * 255
    target[0:new_size[1], 0:new_size[0]] = img
    return target
#     img = cv2.transpose(target)

#     return cv2.transpose(img)


In [60]:
for split in ["test"]:
    split_data = [datum for datum in data if datum['split'] == split]
    gts = []
    imgs = []
    for data in split_data:
        img = cv2.imread(data["file"])
        
        if not img is None:
            gts.append(data["text"])
            img = preprocess(img, (200,100,3))
            imgs.append(img)
            
    print(len(gts))
    with h5py.File("../data/rimes_aug.hdf5", "a") as hf:
        hf.create_dataset(f"{split}/gt", data= gts, compression="gzip", compression_opts=9)        
        hf.create_dataset(f"{split}/dt", data= imgs, compression="gzip", compression_opts=9)
        print(f"[OK] {split} partition.")

7627
[OK] test partition.


In [53]:
print(len(gts))

499999


In [58]:
c = [datum for datum in data if datum['split'] == 'test']

In [59]:
len(c)

7627